In [8]:
!pip install transformers
!pip install datasets transformers[sentencepiece] sacrebleu
import pandas as pd
import numpy as np
from datasets import load_dataset
import tensorflow as tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00


In [9]:
dataset_path = "cfilt/iitb-english-hindi"
train_dataset = load_dataset(dataset_path, split="train[:1000]")

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [10]:
english_sentences = [item['en'] for item in train_dataset['translation']]
hindi_sentences = [item['hi'] for item in train_dataset['translation']]

In [11]:
from transformers import pipeline

pipe = pipeline("translation", model="facebook/mbart-large-50-many-to-many-mmt")

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [12]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

In [13]:
model= MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to('cuda')

tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")

In [14]:
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="hi_IN", tgt_lang='en_XX',max_length=400)

In [17]:
translatedHinEng = []
for (index, (hin, eng)) in enumerate(zip(hindi_sentences, english_sentences)):
    print(index+1)
    print("Hindi:", hin)
    prediction = translator(hin)
    translation = prediction[0]['translation_text']
    print("Translated: ", translation)
    print("Original: ", eng)
    print("------------------------------")
    translatedHinEng.append(translation)

1
Hindi: अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
Translated:  Benefit your application from accessibility exercises
Original:  Give your application an accessibility workout
------------------------------
2
Hindi: एक्सेर्साइसर पहुंचनीयता अन्वेषक
Translated:  Cursor Accessibility Explorer
Original:  Accerciser Accessibility Explorer
------------------------------
3
Hindi: निचले पटल के लिए डिफोल्ट प्लग-इन खाका
Translated:  Default plug - in for bottom panel
Original:  The default plugin layout for the bottom panel
------------------------------
4
Hindi: ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
Translated:  Default plug - in for top panel
Original:  The default plugin layout for the top panel
------------------------------
5
Hindi: उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
Translated:  List of plugins that have been disabled by default
Original:  A list of plugins that are disabled by default
------------------------------
6
Hindi: अवधि को हाइलाइट रकें
Translat

In [18]:
comaparision = pd.DataFrame({'Original': hindi_sentences, 'Translated': translatedHinEng})
comaparision.to_csv('/content/IITB_Translated_2to3.csv', index=False)

In [19]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
references = [[english] for english in english_sentences]
candidates = translatedHinEng
bleu_score = corpus_bleu(references, candidates)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.6810
